In [1]:
from gekko import GEKKO

In [2]:
import numpy as np

In [3]:
# Инициализация модели
m = GEKKO(remote=False)

In [4]:
# Решатель APOPT
m.options.SOLVER = 1

In [5]:
R = np.array([0, 0, 1, 0, 0])

In [6]:
Erf = np.array([[  1,   0,   0,   0],
                [0.5, 0.5,   0,   0],
                [  0, 0.5, 0.5,   0],
                [  0,   0, 0.5, 0.5],
                [  0,   0,   0,   1]])

In [7]:
Eff = np.array([[0, 0, 0, 0],
                [1, 0, 0, 0], # на файл № 2 имеется зависимость у файла № 1
                [0, 0, 0, 0],
                [0, 0, 0, 0]])

In [8]:
Efp = m.Array(m.Var, (4, 3), integer=True)

In [9]:
for row in Efp:
    for el in row:
        el.lower = 0
        el.upper = 1

In [10]:
Efp[0][0].value, Efp[0][1].value, Efp[0][2].value = 1, 0, 0
Efp[1][0].value, Efp[1][1].value, Efp[1][2].value = 0, 1, 0
Efp[2][0].value, Efp[2][1].value, Efp[2][2].value = 0, 1, 0
Efp[3][0].value, Efp[3][1].value, Efp[3][2].value = 0, 0, 1

In [11]:
equations = []
for row in Efp:
    equation = m.sum(row) == 1
    equations.append(equation)

In [12]:
m.Equations(equations)

In [13]:
activate_f = lambda x: 1 if x > 0 else 0

In [14]:
activate_p = lambda x: 1 if x > 0 else 0

In [15]:
activate_r = lambda x: 1 if x == 1 else 0

In [16]:
calc_dependencies = lambda x: np.dot(Eff, F) if x == 0 else np.dot(Eff, calc_dependencies(x - 1))

In [17]:
F = np.dot(R, Erf)

In [18]:
files_count = len(F)
for i in range(files_count):
    dependencies = calc_dependencies(i)
    F += dependencies

In [19]:
F = np.array([activate_f(f) for f in F])

In [20]:
P = np.dot(F, Efp)

In [21]:
for p in P:
    p.value = activate_p(p.value)

In [22]:
F_d = np.dot(Efp, P)

In [23]:
for f_d in F_d:
    f_d.value = activate_f(f_d.value)

In [24]:
R_d = np.dot(Erf, F_d)

In [25]:
for r_d in R_d:
    r_d.value = activate_r(r_d.value)

In [26]:
K_un = (m.sum(R_d) - sum(R)) / (m.sum(R_d))

In [27]:
m.Minimize(K_un)

In [28]:
m.solve()

 ----------------------------------------------------------------
 APMonitor, Version 1.0.3
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :           12
   Intermediates:            0
   Connections  :            0
   Equations    :            5
   Residuals    :            5
 
 Number of state variables:             12
 Number of total equations: -            4
 Number of slack variables: -            0
 ---------------------------------------
 Degrees of freedom       :              8
 
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I:  0 Tm:      0.00 NLPi:    6 Dpth:    0 Lvs:    3 Obj:  6.62E-01 Gap:       NaN
--Integer Solution:   7.50E-01 Lowest Leaf:   6.62E-01 Gap:   8.80E-02
Iter

In [29]:
Efp

array([[[0.0], [0.0], [1.0]],
       [[1.0], [0.0], [0.0]],
       [[0.0], [1.0], [0.0]],
       [[0.0], [0.0], [1.0]]], dtype=object)